In [1]:
!pip3 install natsort

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from natsort import natsorted
import os
import glob
import cv2
import pydicom
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K

In [3]:
train = pd.read_csv('/kaggle/input/rsna-dataframe/train_data.csv')
train = train.astype({'study_id':'str','series_id':'str','instance_number':'str'})
train_des = pd.read_csv('/kaggle/input/rsna-dataframe/train_des.csv')
train_des = train_des.astype({'study_id':'str','series_id':'str'})

In [4]:
train_des

,study_id,series_id,series_description
0,100206310,1792451510,Sagittal T2/STIR
1,100206310,2092806862,Sagittal T1
2,100206310,1012284084,Axial T2
3,1002894806,801316590,Sagittal T2/STIR
4,1002894806,866293114,Sagittal T1
...,...,...,...
5362,991428866,2954790819,Sagittal T1
5363,991428866,3160509931,Axial T2
5364,992674144,1576603050,Sagittal T2/STIR
5365,992674144,1814811290,Sagittal T1


In [5]:
base_dir = '/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/train_images'
AxialT2 = train_des[train_des['series_description']=='Axial T2'].reset_index(drop=True)
AxialT2_path = base_dir + '/' + AxialT2['study_id'] + '/' + AxialT2['series_id'] + '/*.dcm'

In [6]:
def load_images(dir_path, s=0, ends=100):
    
    images = []
    path_list = []
    
    for p in tqdm(dir_path[s:ends]):
        path = natsorted(glob.glob(p))
        path_list += path
        
        for im in path:
            dcm = pydicom.dcmread(im)
            dcm = dcm.pixel_array
            dcm = (dcm - dcm.min()) / (dcm.max() - dcm.min() + 1e-6)*255.0
            nonzero = dcm[dcm > 0]
            dcm = np.clip(dcm, 0, 2*np.std(nonzero)+np.mean(nonzero))
            dcm = dcm / 255.0
            dcm = cv2.resize(dcm,(224,224))
            images.append(dcm)
            
    base_label = base_dir + '/' + train['study_id'] + '/' + train['series_id'] + '/' + train['instance_number']+'.dcm'
    base_label = base_label.unique()
    
    labels = []
    lens = len(path_list)
    
    for i in range(lens):
        te = path_list[i] in base_label
        labels.append(te)
        
    enc = OneHotEncoder(sparse_output=False)
    label = enc.fit_transform(np.array(labels).reshape(-1,1))
            
    return np.array(images)[:,:,:,np.newaxis], label

In [7]:
train_image, train_label = load_images(AxialT2_path, 0, 500)
valid_image, valid_label = load_images(AxialT2_path, 600,700)

100%|██████████| 100/100 [01:21<00:00,  1.22it/s]


In [8]:
def dense_block(x, num_layers, growth_rate):
    for i in range(num_layers):
        # Bottleneck Layer
        bn = layers.BatchNormalization()(x)
        relu = layers.ReLU()(bn)
        conv = layers.Conv2D(4 * growth_rate, (1, 1), padding='same', kernel_initializer='he_normal')(relu)

        # Composite Function
        bn = layers.BatchNormalization()(conv)
        relu = layers.ReLU()(bn)
        conv = layers.Conv2D(growth_rate, (3, 3), padding='same', kernel_initializer='he_normal')(relu)
        
        # Concatenate input with output of this layer
        x = layers.Concatenate()([x, conv])
    return x

def transition_layer(x, reduction):
    # Batch Normalization and ReLU
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    
    # 1x1 Convolution
    num_filters = int(K.int_shape(x)[-1] * reduction)
    x = layers.Conv2D(num_filters, (1, 1), padding='same', kernel_initializer='he_normal')(x)
    
    # Average Pooling
    x = layers.AveragePooling2D((2, 2), strides=(2, 2))(x)
    
    return x

def build_densenet(inputs, depth, growth_rate, reduction=0.5):
    
    # 初期コンボリューションレイヤー
    x = layers.Conv2D(2 * growth_rate, (7, 7), strides=(2, 2), padding='same', kernel_initializer='he_normal')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
    
    # Dense BlockとTransition Layerの構築
    num_dense_blocks = (depth - 4) // 6
    for i in range(3):  # Typically, DenseNet has 3 dense blocks.
        x = dense_block(x, num_dense_blocks, growth_rate)
        if i != 2:  # 最後のDense Blockの後にTransition Layerは不要
            x = transition_layer(x, reduction)
    
    # グローバル平均プーリングと全結合レイヤー
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.GlobalAveragePooling2D()(x)
    
    return x

In [9]:
def CNN_model(input_shape, num_classes, depth, growth_rate):
    
    inputs = Input(shape=input_shape)
    x = build_densenet(inputs, depth, growth_rate)
    
    outputs = layers.Dense(num_classes, activation='sigmoid', name='output')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    
    return model

In [10]:
# モデルの構築
input_shape = (224, 224, 1)  # 画像のサイズ
num_classes = 2  # クラス数
depth = 40  # ネットワークの深さ
growth_rate = 12  # グロースレート

In [11]:
ss_model = CNN_model(input_shape, num_classes, depth, growth_rate)
ss_model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [12]:
ss_history = ss_model.fit(train_image, train_label, 
                            epochs=20, batch_size=64,
                            validation_data=(valid_image, valid_label))

Epoch 1/20


I0000 00:00:1729003457.211764      74 service.cc:145] XLA service 0x7fa7940029d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729003457.211857      74 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  2/280 ━━━━━━━━━━━━━━━━━━━━ 22s 79ms/step - accuracy: 0.1445 - loss: 0.8073   

I0000 00:00:1729003484.217685      74 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - accuracy: 0.7885 - loss: 0.4648

I0000 00:00:1729003528.149879      73 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'loop_add_multiply_reduce_subtract_fusion_2', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_multiply_reduce_subtract_fusion_10', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_multiply_reduce_subtract_fusion_4', 8 bytes spill stores, 8 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'loop_add_multiply_reduce_subtract_fusion_8', 8 bytes spill stores, 8 bytes spill loads



280/280 ━━━━━━━━━━━━━━━━━━━━ 101s 187ms/step - accuracy: 0.7886 - loss: 0.4645 - val_accuracy: 0.8293 - val_loss: 0.6167
Epoch 2/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 24s 75ms/step - accuracy: 0.8641 - loss: 0.3036 - val_accuracy: 0.8508 - val_loss: 0.3538
Epoch 3/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.8814 - loss: 0.2629 - val_accuracy: 0.8892 - val_loss: 0.2613
Epoch 4/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.8925 - loss: 0.2508 - val_accuracy: 0.8835 - val_loss: 0.2912
Epoch 5/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.8974 - loss: 0.2306 - val_accuracy: 0.8833 - val_loss: 0.2548
Epoch 6/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.9023 - loss: 0.2275 - val_accuracy: 0.8890 - val_loss: 0.2478
Epoch 7/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.9051 - loss: 0.2142 - val_accuracy: 0.8796 - val_loss: 0.2699
Epoch 8/20
280/280 ━━━━━━━━━━━━━━━━━━━━ 21s 75ms/step - accuracy: 0.9101 - loss: 0.2074 - val_accur

In [13]:
test_image, test_label = load_images(AxialT2_path, 1200, 1300)

100%|██████████| 100/100 [01:09<00:00,  1.45it/s]


In [14]:
pred = ss_model.predict(test_image)
ac = np.argmax(pred, axis=1)
test = np.argmax(test_label, axis=1)

115/115 ━━━━━━━━━━━━━━━━━━━━ 8s 39ms/step


In [15]:
a = metrics.Accuracy()
a.update_state(ac,test)
print(a.result())

l = metrics.BinaryCrossentropy()
l.update_state(pred,test_label)
print(l.result())

tf.Tensor(0.8628681, shape=(), dtype=float32)
tf.Tensor(2.3957305, shape=(), dtype=float32)


In [16]:
ss_model.save('/kaggle/working/ss_extraction_model.h5')